# practice 1 : probe

For this pratice you will need to have a look at

  * [probeinterface documentation](https://probeinterface.readthedocs.io/en/main/)
  * [probeinterface examples](https://probeinterface.readthedocs.io/en/main/examples/index.html)


In [43]:
#%matplotlib inline
%matplotlib widget

In [44]:
import probeinterface as pi
from probeinterface.plotting import plot_probe

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


## Some probe are already implemented (cambridge neurotec)

In [45]:
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-H6'

probe = pi.get_probe(manufacturer, probe_name)
print(probe)

cambridgeneurotech - ASSY-236-H6 - 64ch


In [46]:
plot_probe(probe, with_contact_id=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba4c06e7c0>,
 <matplotlib.collections.PolyCollection at 0x7fba47f8f700>)

## Let implement neuronexus A1x32-Poly2-10mm-50s-177 manually

  * https://www.neuronexus.com/files/catalog/2021-Probe-Catalog.pdf



## Step 1 : construct a probe from channel position

using the `Probe()` object and some methods:
  * `Probe.set_contacts()`
  * `Probe.set_contact_ids()`
and using the file **'A1x32-Poly2-10mm-50s-177.csv'** let construct the probe.

Then plot it with 
`plot_probe()`
use the `with_contact_id=True` option


In [47]:
df = pd.read_csv('A1x32-Poly2-10mm-50s-177.csv')
df

,contact_id,x,y
0,1,0.0,300
1,2,0.0,350
2,3,0.0,400
3,4,0.0,450
4,5,0.0,500
5,6,0.0,550
6,7,0.0,600
7,8,0.0,650
8,9,0.0,700
9,10,0.0,750


In [48]:

positions = df[['x', 'y']].values
probe = pi.Probe(ndim=2, si_units='um')
probe.set_contacts(positions=positions, shapes='circle', shape_params={'radius': 7.5})
probe.set_contact_ids(df['contact_id'].values)

In [49]:
plot_probe(probe, with_contact_id=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba4c06ecd0>, None)

## Step 2 : set contour

Contour can be set :
  * automatiocally with dummy shape `probe.create_auto_shape()`
  * or manuualy with `probe.set_planar_contour'()`


here the polygon shape of our probe : 

`contour_polygon = []`


In [50]:
probe.create_auto_shape()
plot_probe(probe, with_contact_id=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba4c858c70>,
 <matplotlib.collections.PolyCollection at 0x7fba4c85db20>)

In [51]:
contour_polygon = [[-25, 800],
                   [-11, 0],
                   [43.3/2, -75.],
                   [54.3, 0],
                   [68.3, 800]]
probe.set_planar_contour(contour_polygon)
plot_probe(probe, with_contact_id=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba47dc1d60>,
 <matplotlib.collections.PolyCollection at 0x7fba47dde250>)

## Step 3 : save "probe wired" into json

Using the function `write_probeinterface()` save the probe to a file.

Inspect the file.

In [52]:
pi.write_probeinterface('A1x32-Poly2-10mm-50s-177_unwired.json', probe)

In [53]:
!head -25 A1x32-Poly2-10mm-50s-177_unwired.json

{
    "specification": "probeinterface",
    "version": "0.2.5",
    "probes": [
        {
            "ndim": 2,
            "si_units": "um",
            "annotations": {
                "name": ""
            },
            "contact_annotations": {},
            "contact_positions": [
                [
                    0.0,
                    300.0
                ],
                [
                    0.0,
                    350.0
                ],
                [
                    0.0,
                    400.0
                ],
                [


## Step 4 : wire to device channel (aka pathway)

Now lets do the "wiring" aka channel mapping.
Lets connect our probe to RHD2132 INtan headstage with the H32 connector.

See this: https://intantech.com/RHD_headstages.html?tabSelect=RHD32ch&yPos=0


<img src="Intan_RDH2132_overview.png" width="400"/>
<img src="Intan_RDH2132_connector_pineout.png" width="400"/>

And also, note that the mapping depend on the connector of the probe, see this
https://www.neuronexus.com/files/probemapping/32-channel/H32-Maps.pdf

Probeinterface have 2 ways to make the mapping:

 1. Manually with : `probe.set_device_channel_indices()`
 2. Automatically with `probe.wiring_to_device()`
 
 
 
Use the `with_contact_id=True` and `with_device_index=True` option for plor_probe.


In [54]:
manual_mapping = [
    16, 17, 18, 20, 21, 22, 31, 30, 29, 27, 26, 25, 24, 28, 23, 19,
    12, 8, 3, 7, 6, 5, 4, 2, 1, 0, 9, 10, 11, 13, 14, 15]
probe.set_device_channel_indices(manual_mapping)
plot_probe(probe, with_contact_id=True, with_device_index=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba47d5bd90>,
 <matplotlib.collections.PolyCollection at 0x7fba47d69f40>)

In [55]:
probe.wiring_to_device('H32>RHD2132')
plot_probe(probe, with_contact_id=True, with_device_index=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<matplotlib.collections.PolyCollection at 0x7fba47ceacd0>,
 <matplotlib.collections.PolyCollection at 0x7fba47c8adf0>)

In [56]:
probe_df = probe.to_dataframe(complete=True)
probe_df[['contact_ids', 'device_channel_indices', 'x', 'y']]

,contact_ids,device_channel_indices,x,y
0,1,16,0.0,300.0
1,2,17,0.0,350.0
2,3,18,0.0,400.0
3,4,20,0.0,450.0
4,5,21,0.0,500.0
5,6,22,0.0,550.0
6,7,31,0.0,600.0
7,8,30,0.0,650.0
8,9,29,0.0,700.0
9,10,27,0.0,750.0


## Step  : save the "probe wired" into json

and inspect file

In [57]:
pi.write_probeinterface('A1x32-Poly2-10mm-50s-177_wired.json', probe)

In [58]:
!head -25 A1x32-Poly2-10mm-50s-177_unwired.json

{
    "specification": "probeinterface",
    "version": "0.2.5",
    "probes": [
        {
            "ndim": 2,
            "si_units": "um",
            "annotations": {
                "name": ""
            },
            "contact_annotations": {},
            "contact_positions": [
                [
                    0.0,
                    300.0
                ],
                [
                    0.0,
                    350.0
                ],
                [
                    0.0,
                    400.0
                ],
                [
